## Wczytanie danych i bibliotek

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# file = "ks_projects_201801.csv"
# filepath = os.path.join(os.path.abspath(""), file)

In [ ]:
df = pd.read_csv("ks_projects_201801.csv")


## Wstępne sprawdzenie danych

In [ ]:
# Usunięcie wartości NaN
df = df.dropna()

# Ujednolicenie kolumn z datami, przekształcenie na format datetime
df['deadline_new'] = pd.to_datetime(df['deadline']).dt.date
df['launched_new'] = pd.to_datetime(df['launched']).dt.date

# Obliczenie czasu trwania projektów, oczyszczenie danych - przekształcenie na typ int
df['duration'] = df['deadline_new'] - df['launched_new']
df['duration'] = df['duration'].map(lambda x: int(str(x).split(" ")[0]))

# Wyodrębnienie dwóch stanów projektów: successful = 1, pozostałe = 0
df["state_new"] = df["state"].map(lambda x: 1 if x == "successful" else 0 )


# df.sample(5)

In [ ]:
# df.columns

In [ ]:
# Usunięcie niepotrzebnych kolumn
to_drop = ["currency",
           "deadline",
           "goal",
           "launched",
           "pledged",
           "state",
           "usd pledged"]

df.drop(to_drop, inplace=True, axis=1)

In [ ]:
# df.head()

In [ ]:
# Sprawdzenie, czy projekty się powtarzają
df["ID"].duplicated().any()

## Ogólna analiza danych

In [ ]:
# Sprawdzenie ile było projektów w poszczególnych krajach
project_country = dict(df["country"].value_counts())

# project_country

In [ ]:
# Połączenie Luksemburga, Belgii i Holandii w Beneluks

df["country"] = df["country"].replace(["LU", "BE"],"NL")

project_country = dict(df["country"].value_counts())
# project_country

In [ ]:
# Odrzucenie krajów, w których liczba projektów jest poniżej 1000

df = df[(df["country"] != "SG") 
        & (df["country"] != "JP") 
        & (df["country"] != "AT") 
        & (df["country"] != "HK")
        & (df["country"] != "NO")
        & (df["country"] != "CH")
        & (df["country"] != "IE")]
# df["country"].unique()

In [ ]:
# Zaktualizowana liczba projektów w poszczególnych krajach

project_country_s = dict(df["country"].value_counts())
# project_country_s

In [ ]:
# Odfiltrowanie projektów, które odniosły sukces w poszczególnych krajach

success_filter = df[df["state_new"] == 1]
group_success_filter = success_filter.groupby(by=["country"])
sum_success = dict(group_success_filter["state_new"].sum())

# sum_success

In [ ]:
# Wizualizacja liczby projektów w poszczególnych krajch - sumarycznie i tych, które zdobyły finansowanie

all_country_list = []
success_country_list = []
country_list = []
for k, v in project_country_s.items():
    all_country_list.append(v)
    success_item = sum_success[k]
    success_country_list.append(success_item)
    country_list.append(k)
        
# print(all_country_list)
# print(success_country_list)
# print(country_list)

In [ ]:
sns.set_theme(style="whitegrid")

# Ustawienia ogólne dla wykresu
f, ax = plt.subplots(figsize=(15, 10))
plt.title("Number of projects in selected countries", fontdict={'fontsize': 18})
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Number of projects', fontdict={'fontsize': 16})

# Wykres ogólnej liczby projektów
sns.set(font_scale=1.5)
sns.set_color_codes("pastel")
sns.barplot(x=country_list, y=all_country_list,
            label="Total", color="r")

# Wykres liczby projektów, które uzyskały finansowanie
sns.set_color_codes("muted")
sns.barplot(x=country_list, y=success_country_list,
            label="Succeed", color="r")

# Formatowanie legendy
ax.legend(ncol=2, loc="center right", frameon=True)
sns.despine(left=True, bottom=True)

In [ ]:
# Procent projektów, które się udały

perc_success = {key: round(((sum_success[key] / project_country[key]) * 100), 1) for key, value in sum_success.items()}

# Procent projektów, które się udały - sortowanie wartości

perc_success_sort = {k: v for k, v in sorted(perc_success.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
# Wizualizacja odsetka sfinansowanych projektów w zależności od kraju

perc_country_list = []
country_list_2 = []
for country, perc in perc_success_sort.items():
    perc_country_list.append(perc)
    country_list_2.append(country)
        
# print(perc_country_list)
# print(country_list_2)

In [ ]:
sns.set_theme(style="white", context="talk")

plt.figure(figsize=(15, 5))
plt.title("Percent of successful projects in selected countries", fontdict={'fontsize': 18})
plt.ylabel("Successful projects [%]", fontdict={'fontsize': 16})

sns.barplot(x=country_list_2, y=perc_country_list, palette="magma")

sns.despine(left=True);

#### Charakterystyka rozkładu liczby osób wspierających projekty

In [ ]:
# Charakterystyka rozkładu liczby osób wspierających projekty

df.groupby(by="country")["backers"].describe().style.set_properties(**{'text-align':'center', 'padding':'15px'})

In [ ]:
# Wizualizacja rozkładu liczby osób wspierających projekty

sns.set_theme(style="whitegrid")

# Ogólne ustawienia dla wykresu
fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Distribution of the project's backers", fontdict={'fontsize': 18})

# Wykres skrzynkowy dla rozkładu liczby osób wspierających projekty
sns.boxplot(x=df["backers"], y=df["country"]);

ax.set_xlabel("Backers",fontsize=15)
ax.tick_params(labelsize=15)
ax.set_ylabel("",fontsize=15)
sns.despine(left=True);

In [ ]:
# Usunięcie 5 % najwyższych wartości

df_95b = df[(df["backers"] < np.quantile(df["backers"],0.95))]

In [ ]:
# Wizualizacja rozkładu liczby osób wspierających projekty

sns.set_theme(style="whitegrid")

# Ogólne ustawienia dla wykresu
fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Distribution project's backers ( < 95 %)", fontdict={'fontsize': 18})

# Wykres skrzynkowy dla rozkładu liczby osób wspierających projekty
sns.boxplot(x=df_95b["backers"], y=df_95b["country"]);

ax.set_xlabel("Backers",fontsize=15)
ax.tick_params(labelsize=15)
ax.set_ylabel("",fontsize=15)
sns.despine(left=True);

In [ ]:
sns.set_theme(style="whitegrid")

plt.figure(figsize=(15, 8))
plt.title("Distribution of project's backers ( < 95 %)", fontdict={'fontsize': 18})
plt.xlabel('Backers', fontdict={'fontsize': 15})
plt.ylabel("", fontdict={'fontsize': 15})
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

sns.histplot(df_95b["backers"], bins=100, kde=True, color="indigo");

#### Charakterystyka rozkładu liczby osób wspierających sfinansowane projekty

In [ ]:
# Charakterystyka rozkładu liczby osób wspierających projekty, które uzyskały finansowanie

success_filter.groupby(by="country")["backers"].describe().style.set_properties(**{'text-align':'center', 'padding':'15px'})

In [ ]:
# Wizualizacja rozkładu liczby osób wspierających projekty, które uzyskały finansowanie

sns.set_theme(style="whitegrid")

# Ogólne ustawienia dla wykresu
fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Distribution of the successful project's backers", fontdict={'fontsize': 18})

# Wykres skrzynkowy dla rozkładu liczby osób wspierających projekty
sns.boxplot(x=success_filter["backers"], y=success_filter["country"]);

ax.set_xlabel("Backers",fontsize=15)
ax.tick_params(labelsize=15)
ax.set_ylabel("",fontsize=15)
sns.despine(left=True);

In [ ]:
# Usunięcie 5 % najwyższych wartości

df_95bs = success_filter[(success_filter["backers"] < np.quantile(success_filter["backers"],0.95))]

In [ ]:
# Wizualizacja rozkładu liczby osób wspierających projekty, które uzyskały finansowanie

sns.set_theme(style="whitegrid")

# Ogólne ustawienia dla wykresu
fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Distribution of successful project's backers ( < 95 %)", fontdict={'fontsize': 18})

# Wykres skrzynkowy dla rozkładu liczby osób wspierających projekty
sns.boxplot(x=df_95bs["backers"], y=df_95bs["country"]);

ax.set_xlabel("Backers",fontsize=15)
ax.tick_params(labelsize=15)
ax.set_ylabel("",fontsize=15)
sns.despine(left=True);

In [ ]:
sns.set_theme(style="whitegrid")

plt.figure(figsize=(15, 8))
plt.title("Distribution of successful project's backers ( < 95 %)", fontdict={'fontsize': 18})
plt.xlabel('Backers', fontdict={'fontsize': 15})
plt.ylabel("", fontdict={'fontsize': 15})
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

sns.histplot(df_95bs["backers"], bins=100, kde=True, color="indigo");

## Wyodrębnienie czynników mających wpływ na zakończenie projektu sukcesem

### 1. Miesiąc rozpoczęcia zbiórki

In [ ]:
# Wyodrębnienie roku i miesiąca z danych 

df['year_launched'] = pd.DatetimeIndex(df['launched_new']).year
df['month_launched'] = pd.DatetimeIndex(df['launched_new']).month
df['year_deadline'] = pd.DatetimeIndex(df['deadline_new']).year
df['month_deadline'] = pd.DatetimeIndex(df['deadline_new']).month
# df.head()

In [ ]:
# Sprawdzenie ile projektów powstało w poszczególnych latach

df["year_launched"].value_counts()

In [ ]:
# Usunięcie rekordów z 1970 roku

df = df[df["year_launched"] != 1970]
df["year_launched"].unique()

In [ ]:
# Ile było rozpoczętych projektów w poszczególnych krajach w danym miesiącu

all_month_grouped = df.groupby(by=["country", "month_launched"]).size().reset_index(name="sum")

# Usuwam US z danych do Heatmap ze względu na to, że znacznie zawyża górną granicę danych (nie widać różnic w innych krajach)
all_month_grouped_us = all_month_grouped[all_month_grouped["country"] != "US"]


In [ ]:
# Przygotowanie danych do Heatmap'y

df_month = all_month_grouped_us.pivot("country", "month_launched", "sum")
# df_month

In [ ]:
# Wizualizacja liczby projektów w analizowanych krajach w poszczególnych miesiącach

fig, ax = plt.subplots(figsize=(15, 8))

plt.title("No. of projects", fontdict={'fontsize': 18})

sns.heatmap(df_month, fmt="d", annot=True, annot_kws={"fontsize":15}, linewidths=.5, cmap="RdPu")

ax.set_xlabel("Months",fontsize=15)
ax.tick_params(labelsize=15)
ax.set_ylabel("Countries",fontsize=15);

In [ ]:
# Odfiltrowanie projektów, które uzyskały finansowanie

success_filter = df[df["state_new"] == 1]

month_grouped = success_filter.groupby(by=["country", "month_launched"]).size().reset_index(name="sum_success")

In [ ]:
# Stworzenie tabeli z pogrupowanymi projektami

df_merge_month = pd.merge(all_month_grouped, month_grouped, on=["country", "month_launched"], how="left")

df_merge_month["perc_success"] = round(100 * df_merge_month["sum_success"] / df_merge_month["sum"], 2)

In [ ]:
# Przygotowanie danych do Heatmap'y

df_month_perc = df_merge_month.pivot("country", "month_launched", "perc_success")

In [ ]:
# Wizualizacja liczby sfinansowanych projektów w analizowanych krajach w poszczególnych miesiącach

fig, ax = plt.subplots(figsize=(15, 8))

plt.title("No. of funded projects", fontdict={'fontsize': 18})

sns.heatmap(df_month_perc, annot=True, annot_kws={"fontsize":15}, linewidths=.5, cmap="RdPu")

ax.set_xlabel("Months",fontsize=15)
ax.tick_params(labelsize=15)
ax.set_ylabel("Countries",fontsize=15);

#### W związku z tym, że nie ma wyraźnego trendu wskazującego na to, że w jakimś okresie szansa na uzyskanie finansowania projektu  jest większa (zarówno najmniejsze jak i największe wartości są rozdystrybułowane dość przypadkowo), nie bierzemy tej zależności pod uwagę w budowaniu aplikacji.

### 2. Kategoria główna

In [ ]:
df["main_category"].unique()

In [ ]:
# Ile było projektów w poszczególnych krajach w danej kategorii

all_mcategory_grouped = df.groupby(by=["country", "main_category"]).size().reset_index(name="sum")

# Ile było projektów, które zostały sfinansowane w poszczególnych krajach w danej kategorii

mcategory_grouped = success_filter.groupby(by=["country", "main_category"]).size().reset_index(name="sum_success")

# Stworzenie tabeli z pogrupowanymi projektami i odsetkiem sfinansowanych projektów

df_merge_mcategory = pd.merge(all_mcategory_grouped, mcategory_grouped, on=["country", "main_category"], how="left")
df_merge_mcategory = df_merge_mcategory.fillna(0.0)
df_merge_mcategory["perc_success"] = round(100 * df_merge_mcategory["sum_success"] / df_merge_mcategory["sum"], 2)

In [ ]:
# Wizualizacja odsetka sfinansowanych projektów w zależności od kategorii, w której projekt został złożony
sns.set_theme(style="white", context="talk")

fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Percent of successful projects in selected main categories (US)", fontdict={'fontsize': 18})

df_country = df_merge_mcategory[df_merge_mcategory["country"] == "US"]b

sns.barplot(x="main_category", y="perc_success", palette="magma", data=df_country)

ax.set_xlabel("",fontsize=15)
ax.tick_params(labelsize=15, rotation=90)
ax.set_ylabel("Percent of founded projects [%]",fontsize=15)
sns.despine(left=True);

In [ ]:
# Wizualizacja odsetka sfinansowanych projektów w zależności od kategorii, w której projekt został złożony
sns.set_theme(style="white", context="talk")

fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Percent of successful projects in selected main categories (GB)", fontdict={'fontsize': 18})

df_country = df_merge_mcategory[df_merge_mcategory["country"] == "GB"]

sns.barplot(x="main_category", y="perc_success", palette="magma", data=df_country)

ax.set_xlabel("",fontsize=15)
ax.tick_params(labelsize=15, rotation=90)
ax.set_ylabel("Percent of founded projects [%]",fontsize=15)
sns.despine(left=True);

In [ ]:
# Wizualizacja odsetka sfinansowanych projektów w zależności od kategorii, w której projekt został złożony
sns.set_theme(style="white", context="talk")

fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Percent of successful projects in selected main categories (IT)", fontdict={'fontsize': 18})

df_country = df_merge_mcategory[df_merge_mcategory["country"] == "IT"]

sns.barplot(x="main_category", y="perc_success", palette="magma", data=df_country)

ax.set_xlabel("",fontsize=15)
ax.tick_params(labelsize=15, rotation=90)
ax.set_ylabel("Percent of founded projects [%]",fontsize=15)
sns.despine(left=True);

In [ ]:
# Wizualizacja odsetka sfinansowanych projektów w zależności od kategorii, w której projekt został złożony
sns.set_theme(style="white", context="talk")

fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Percent of successful projects in selected main categories (MX)", fontdict={'fontsize': 18})

df_country = df_merge_mcategory[df_merge_mcategory["country"] == "MX"]

sns.barplot(x="main_category", y="perc_success", palette="magma", data=df_country)

ax.set_xlabel("",fontsize=15)
ax.tick_params(labelsize=15, rotation=90)
ax.set_ylabel("Percent of founded projects [%]",fontsize=15)
sns.despine(left=True);

#### We wszystkich państwach widać wyraźną zależność powodzenia projektu od tego w jakiej kategorii został złożony, więc kategoria będzie jednym z parametrów analizowanych w aplikacji.

### 3. Podkategoria

In [ ]:
# df["category"].unique()

In [ ]:
# Ile było projektów w poszczególnych krajach w danej podkategorii

all_category_grouped = df.groupby(by=["country", "main_category", "category"]).size().reset_index(name="sum")

# Ile było projektów, które zostały sfinansowane w poszczególnych krajach w danej podkategorii

category_grouped = success_filter.groupby(by=["country", "main_category", "category"]).size().reset_index(name="sum_success")

# Stworzenie tabeli z pogrupowanymi projektami i odsetkiem sfinansowanych projektów

df_merge_category = pd.merge(all_category_grouped, category_grouped, on=["country", "main_category", "category"], how="left")
df_merge_category = df_merge_category.fillna(0.0)
df_merge_category["perc_success"] = round(100 * df_merge_category["sum_success"] / df_merge_category["sum"], 2)

# df_merge_category

In [ ]:
# Wizualizacja odsetka sfinansowanych projektów w zależności od podkategorii, w której projekt został złożony

df_country_cat = df_merge_category[ df_merge_category["country"] == "US" ]

sns.set_theme(style="whitegrid")

fig, ax = plt.subplots(figsize=(6, 35))

plt.title("Percent of successful projects in selected categories", fontdict={'fontsize': 18})

sns.barplot(x="perc_success", y="category", hue="main_category", data=df_country_cat, dodge=False)

ax.legend(ncol=1, loc="lower right", frameon=True)
ax.set_xlabel("Percent of founded projects [%]",fontsize=15)
ax.tick_params(labelsize=12)
ax.set_ylabel("",fontsize=20)
sns.despine(left=True, bottom=True);

#### We wszystkich państwach widać wyraźną zależność powodzenia projektu od tego w jakiej podkategorii został złożony, więc podkategoria jest jednym z parametrów analizowanych w aplikacji.

### 4. Czas trwania zbiórki

In [ ]:
# Charakterystyka rozkładu czasu trwania projektów

df["duration"].describe().to_frame().style.set_properties(**{'padding':'15px'})

In [ ]:
# Wizualizacja czasu trwania projektów w róznych krajach

sns.set_theme(style="whitegrid")

# Ogólne ustawienia dla wykresu
fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Duration of the project", fontdict={'fontsize': 18})

# Wykres skrzynkowy dla rozkładu czasu trwania projektów
sns.boxplot(x=df["duration"], y=df["country"]);

ax.set_xlabel("Days",fontsize=15)
ax.tick_params(labelsize=15)
ax.set_ylabel("Countries",fontsize=15)
sns.despine(left=True);

In [ ]:
sns.set_theme(style="whitegrid")

plt.figure(figsize=(15, 8))
plt.title("Duration of the project", fontdict={'fontsize': 18})
plt.xlabel('Days', fontdict={'fontsize': 15})
plt.ylabel('No. of projects', fontdict={'fontsize': 15})
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

sns.histplot(df["duration"], bins=25, color="indigo");

In [ ]:
# Charakterystyka rozkładu czasu trwania sfinansowanych projektów

success_filter["duration"].describe().to_frame().style.set_properties(**{'padding':'15px'})

In [ ]:
# Wizualizacja czasu trwania sfinansowanych projektów

sns.set_theme(style="whitegrid")

# Ogólne ustawienia dla wykresu
fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Duration of the successful projects", fontdict={'fontsize': 18})

# Wykres skrzynkowy dla rozkładu czasu trwania projektów
sns.boxplot(x=success_filter["duration"], y=df["country"]);

ax.set_xlabel("Days",fontsize=15)
ax.tick_params(labelsize=15)
ax.set_ylabel("Countries",fontsize=15)
sns.despine(left=True);

#### Możliwe jest określenie przedziałów czasowych, które mogłyby zwiększyć prawdopodobieństwo zakończenia zbiórki sukcesem

### 5. Kwota zbiórki

#### Charakterystyka rozkładu kwot zgłaszanych zbiórek

In [ ]:
# Charakterystyka rozkładu kwot docelowych zgłaszanych projektów

df.groupby(by="country")["usd_goal_real"].describe().style.set_properties(**{'text-align':'center', 'padding':'15px'})

In [ ]:
# Wizualizacja rozkładu kwot docelowych zgłaszanych projektów
sns.set_theme(style="whitegrid")

# Ogólne ustawienia dla wykresu
fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Distribution of the project's target costs", fontdict={'fontsize': 18})

# Wykres skrzynkowy dla rozkładu kwot projektów w wybranych krajach
sns.boxplot(x=df["usd_goal_real"], y=df["country"]);

ax.set_xlabel("Goal [$]",fontsize=15)
ax.tick_params(labelsize=15)
ax.set_ylabel("",fontsize=15)
sns.despine(left=True);

In [ ]:
# Usunięcie 5 % najwyższych wartości

df_95 = df[(df['usd_goal_real'] < np.quantile(df['usd_goal_real'],0.95))]

In [ ]:
# Wizualizacja rozkładu kwot docelowych zgłaszanych projektów
sns.set_theme(style="whitegrid")

# Ogólne ustawienia dla wykresu
fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Distribution of the project's target costs ( < 95 %)", fontdict={'fontsize': 18})

# Wykres skrzynkowy dla rozkładu kwot projektów w wybranych krajach
sns.boxplot(x=df_95["usd_goal_real"], y=df_95["country"]);

ax.set_xlabel("Goal [$]",fontsize=15)
ax.tick_params(labelsize=15)
ax.set_ylabel("",fontsize=15)
sns.despine(left=True);

#### Charakterystyka rozkładu kwot sfinansowanych zbiórek

In [ ]:
# Charakterystyka rozkładu kwot zebranych zgłaszanych projektów

df.groupby(by="country")["usd_pledged_real"].describe().style.set_properties(**{'text-align':'center', 'padding':'15px'})

In [ ]:
# Wizualizacja rozkładu kwot zebranych zgłaszanych projektów

sns.set_theme(style="whitegrid")

# Ogólne ustawienia dla wykresu
fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Distribution of the collected amount of money", fontdict={'fontsize': 18})

# Wykres skrzynkowy dla rozkładu zebranych kwot projektów w wybranych krajach
sns.boxplot(x=df["usd_pledged_real"], y=df["country"]);

ax.set_xlabel("Collected [$]",fontsize=15)
ax.tick_params(labelsize=15)
ax.set_ylabel("",fontsize=15)
sns.despine(left=True);

In [ ]:
# Usunięcie 5 % najwyższych wartości

df_95p = df[(df['usd_pledged_real'] < np.quantile(df['usd_pledged_real'],0.95))]

In [ ]:
# Wizualizacja rozkładu kwot zebranych zgłaszanych projektów

sns.set_theme(style="whitegrid")

# Ogólne ustawienia dla wykresu
fig, ax = plt.subplots(figsize=(15, 5))

plt.title("Distribution of the collected amount of money ( < 95 %)", fontdict={'fontsize': 18})

# Wykres skrzynkowy dla rozkładu zebranych kwot projektów w wybranych krajach
sns.boxplot(x=df_95p["usd_pledged_real"], y=df_95p["country"]);

ax.set_xlabel("Collected [$]",fontsize=15)
ax.tick_params(labelsize=15)
ax.set_ylabel("",fontsize=15)
sns.despine(left=True);

#### Zadeklarowaną kwotę projektu należy uznać za czynnik mający wpływ na jego powodzenie